# Simulations with OSN adaptation

Same model of OSN saturation as in ``nonlinear_osn_turbulent_illustration.ipynb``, 

$$ s_i(t) = F_\mathrm{max} \frac{\sum_\gamma K_{i \gamma} c_\gamma}{\exp{(\epsilon_i(t))} + \sum_\gamma K_{i \gamma} c_\gamma} $$

but with modified IBCM and BioPCA integration functions that promote $\epsilon_i(t)$ to dynamical variables with feedback from OSN activity $s_i(t)$ and a target amplitude

$$ \frac{\mathrm{d} \epsilon_i(t)}{\mathrm{d} t} = \frac{1}{\tau_\mathrm{a}} \left( s_i(t) - s_{i, 0} \right) $$

where we also clip (i.e. stop updating beyond this range) $\epsilon \in [\epsilon_L, \epsilon_H]$ to prevent divergences arising from a continued excess or deficit of OSN activity. In other words, adaptation only occurs on a finite range.  


## Functions of general interest

In [ ]:
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from time import perf_counter
import json
from os.path import join as pj
import sys
if "../" not in sys.path:
    sys.path.insert(1, "../")

from modelfcts.ibcm import (
    ibcm_respond_new_odors,  # Unchanged if we give inputs nonlinearized with correct epsilon
    compute_mbars_hgammas_hbargammas,  # Unchanged
)
from modelfcts.biopca import (
    build_lambda_matrix,  
    biopca_respond_new_odors  # Unchanged if we give nonlinearized inputs with correct epsilon
)
# Do not consider average or idealized subtraction here
from modelfcts.ideal import (
    compute_optimal_matrix_fromsamples
)
from modelfcts.checktools import (
    analyze_pca_learning  # Unchanged if give pre-computed nonlinear inputs
)
from utils.metrics import jaccard, l2_norm
from modelfcts.distribs import (
    truncexp1_average,
    powerlaw_cutoff_inverse_transform,
    inverse_transform_tanhcdf
)
# re-use functions for nonlinear OSNs, will need to put 
# updated epsilon in back_params at each step
from modelfcts.nonlin_adapt_osn import (  
    generate_odor_tanhcdf, 
    combine_odors_affinities, 
    update_powerlaw_times_concs_affinities,
)
from modelfcts.backgrounds import (  #
    logof10, 
    sample_ss_conc_powerlaw,   # unchanged
)
from modelfcts.tagging import (  # unchanged
    project_neural_tag, 
    create_sparse_proj_mat, 
    SparseNDArray, 
)
from utils.statistics import seed_from_gen
from utils.smoothing_function import moving_average

from simulfcts.plotting import (
    plot_hbars_gamma_series, 
    plot_w_matrix, 
    plot_background_norm_inhibition, 
    plot_pca_results, 
    hist_outline
)
from simulfcts.analysis import compute_back_reduction_stats

# Initialization

In [ ]:
do_save_plots = False
do_save_outputs = False

root_dir = pj("..")
outputs_folder = pj(root_dir, "results", "for_plots", "nonlin_adapt")
panels_folder = pj(root_dir, "figures", "nonlin_adapt")
params_folder = pj(root_dir, "results", "common_params")

# rcParams
with open(pj(params_folder, "olfaction_rcparams.json"), "r") as f:
    new_rcParams = json.load(f)
plt.rcParams.update(new_rcParams)

# color maps
with open(pj(params_folder, "back_colors.json"), "r") as f:
    all_back_colors = json.load(f)
back_color = all_back_colors["back_color"]
back_color_samples = all_back_colors["back_color_samples"]
back_palette = all_back_colors["back_palette"]

with open(pj(params_folder, "orn_colors.json"), "r") as f:
    orn_colors = json.load(f)
    
with open(pj(params_folder, "inhibitory_neuron_two_colors.json"), "r") as f:
    neuron_colors = np.asarray(json.load(f))
with open(pj(params_folder, "inhibitory_neuron_full_colors.json"), "r") as f:
    neuron_colors_full24 = np.asarray(json.load(f))
# Here, 32 neurons, need to make a new palette with same parameters
neuron_colors_full = np.asarray(sns.husl_palette(n_colors=32, h=0.01, s=0.9, l=0.4, as_cmap=False))

with open(pj(params_folder, "model_colors.json"), "r") as f:
    model_colors = json.load(f)
with open(pj(params_folder, "model_nice_names.json"), "r") as f:
    model_nice_names = json.load(f)

models = list(model_colors.keys())
print(models)
    
models = list(model_colors.keys())

# Main new simulation functions

See functions ``integrate_ibcm_adaptation`` in ``modelfcts.ibcm`` and ``integrate_biopca_adaptation`` in ``modelfcts.biopca``. 

In [ ]:
from modelfcts.ibcm import integrate_ibcm_adaptation
from modelfcts.biopca import integrate_biopca_adaptation

# Parameters

## Common parameters

In [ ]:
# Initialize common simulation parameters
n_dimensions = 50  # Fly number
n_components = 6  # Number of background odors

# Common parameters for toy and full simulations
inhib_rates = [0.00005, 0.00001]  # alpha, beta  [0.00025, 0.00005]

# Simulation duration
duration = 360000.0
deltat = 1.0

# Simulation skipping, 50 is enough for plots
skp = 50 * int(1.0 / deltat)
tser_common = np.arange(0.0, duration, deltat*skp)

# Common model options
activ_function = "identity"  #"ReLU"

# Background process
combine_fct = combine_odors_affinities
update_fct = update_powerlaw_times_concs_affinities

# Scale of affinity vectors: default
kscale = 5e-4  # default is 5e-4

# OSN target activity and epsilon ranges
# TODO; maybe adjust given the odor vectors, f_max scale, etc. 
target_osn_activ = np.full(n_dimensions, 1.0 / np.sqrt(n_dimensions))
adaptation_params = [
    25.0,  # tau_adapt = 250 ms
    1.0,  # eps_min, allow quite low
    10.0,  # eps_max
    target_osn_activ 
]

## Background initialization functions

In [ ]:
def default_background_params(n_comp):
    """ Default time and concentration parameters for the turbulent process"""
    # Turbulent background parameters: same rates and constants for all odors
    back_pms_turbulent = [
        np.asarray([1.0] * n_comp),        # whiff_tmins
        np.asarray([500.] * n_comp),       # whiff_tmaxs
        np.asarray([1.0] * n_comp),        # blank_tmins
        np.asarray([800.0] * n_comp),      # blank_tmaxs
        np.asarray([0.6] * n_comp),        # c0s
        np.asarray([0.5] * n_comp),        # alphas
    ]
    return back_pms_turbulent

In [ ]:
def initialize_back_params(adapt_params, rgen, n_comp, n_dim):
    # Turbulent background parameters: same rates and constants for all odors
    back_pms = default_background_params(n_comp)
    
    tau_eps, eps_min, eps_max, osn_targets = adapt_params
    epsils_vec = np.full(n_dim, 0.5 * (eps_min + eps_max))
    back_comps = generate_odor_tanhcdf((n_comp, n_dim), rgen, unit_scale=kscale)

    # To keep OSN amplitudes comparable to usual simulations, scale down OSN max. ampli
    avg_whiff_conc = np.mean(truncexp1_average(*back_pms[4:6]))
    
    # Same adjustment of the OSN amplitude as in the performance recognition tests
    raw_conc_factor = 2.5
    raw_ampli = 2.5
    np_statistic = np.mean  # np.mean, np.median, np.amax

    raw_osn_activ = np_statistic(combine_fct(np.full(n_comp, raw_conc_factor * avg_whiff_conc), 
                                        back_comps, epsils_vec, fmax=1.0))
    max_osn_ampli = raw_ampli / (raw_osn_activ * np.sqrt(n_dim))

    # Add these extra parameters to the list of background params
    back_pms.append(max_osn_ampli)
    back_pms.append(epsils_vec)
    back_pms.append(back_comps)

    # Initialization
    # Initial values of background process variables (t, c for each variable)
    init_concs = sample_ss_conc_powerlaw(*back_pms[:-3], size=1, rgen=rgen)
    init_times = powerlaw_cutoff_inverse_transform(
                    rgen.random(size=n_comp), *back_pms[2:4])
    tc_init = np.stack([init_times, init_concs.squeeze()], axis=1)

    # Initial background vector: combine odors with the tc_init concentrations
    init_bkvec = combine_fct(tc_init[:, 1], back_comps, epsils_vec, fmax=max_osn_ampli)
    # nus are first in the list of initial background params
    init_back = [tc_init, init_bkvec]
    
    return back_pms, init_back

### Function to run and clean a simulation

Uses global IBCM parameters defined above. 

In [ ]:
def run_ibcm_simulation_adapt(adapt_params, n_comp, n_dim, rgenseed, simseed, skp_local=skp, n_i=None):
    print("Initializing IBCM simulation for adapt_params[:3] =", adapt_params[:3])
    # Initialize background with the random generator with seed rgenseed
    rgen = np.random.default_rng(rgenseed)
    res = initialize_back_params(adapt_params, rgen, n_comp, n_dim)
    back_params_local, init_back = res
    if n_i is None:
        n_i = n_comp * 4
    # Initial synaptic weights: small positive noise
    init_synapses_ibcm = 0.2*rgen.standard_normal(size=[n_i, n_dim])*lambd_ibcm
    
    # Run the IBCM simulation
    print("Starting IBCM simulation...")
    tstart = perf_counter()
    sim_results = integrate_ibcm_adaptation(
                init_synapses_ibcm, update_fct, init_back, 
                ibcm_rates, inhib_rates, back_params_local, 
                adapt_params, duration, deltat, seed=simseed, 
                noisetype="uniform",  skp=skp_local, **ibcm_options
    )
    tend = perf_counter()
    print("Finished IBCM simulation in {:.2f} s".format(tend - tstart))
    
    return back_params_local, sim_results

In [ ]:
def mix_new_odors_in_manifold(back_pms, conc_ser, new_conc_rel, rgen, n_ex=2, n_samp=10):
    """Mix n_ex new odors with n_samp background samples each. 
    Returns a 3d-array of mixtures, indexed [n_ex, n_samp, n_dim], 
    and the new odor vectors, a 2d array indexed [n_ex, n_dim]. 
    """
    back_odors = back_pms[-1]
    n_comp, n_dim = back_odors.shape[0], back_odors.shape[1]
    max_ampli = back_pms[-3]
    new_odors = generate_odor_tanhcdf((n_ex, n_dim), rgen, unit_scale=kscale)
    avg_whiff_conc = np.mean(truncexp1_average(*back_pms[4:6]))
    new_conc = avg_whiff_conc * new_conc_rel
    non_null_concs = conc_ser[np.any(conc_ser > 0.0, axis=1)]
    epsils_vec = back_pms[-2]
    back_concs = non_null_concs[rgen.choice(non_null_concs.shape[0], size=n_ex*n_samp, replace=True)]
    back_concs = back_concs.reshape(n_ex, n_samp, n_comp)
    all_mixed_samples = []
    for i in range(n_ex):
        joint_kmats = np.concatenate([back_odors, new_odors[i:i+1]], axis=0)
        mixed_samples_i = []
        for j in range(n_samp):
            joint_concs = np.concatenate([back_concs[i, j:j+1], np.full((1, 1), new_conc)], axis=1)
            mixed_samples_i.append(combine_fct(joint_concs, joint_kmats, epsils_vec, fmax=max_ampli))
        mixed_samples_i = np.concatenate(mixed_samples_i, axis=0)
        all_mixed_samples.append(mixed_samples_i)
        
    return np.stack(all_mixed_samples, axis=0), new_odors

In [ ]:
# Cleaning function
def analyze_clean_ibcm_simul(results_raw, back_pms, rgenseed, n_ex=2, n_samp=10, t_mix=-1):
    """
    Args:
        results_raw = (tser_ibcm, nuser_ibcm, bkvecser_ibcm, mser_ibcm, 
            cbarser_ibcm, thetaser_ibcm, wser_ibcm, yser_ibcm)
    Returns:
        cbars_gamma, wser_ibcm, bkvecser_ibcm, 
            yser_ibcm, moments_conc, cgammas_bar_counts, specif_gammas, correl_c_conc
    """
    (tser_ibcm, nuser_ibcm, bkvecser_ibcm, eps_ser, mser_ibcm, 
        cbarser_ibcm, thetaser_ibcm, wser_ibcm, yser_ibcm) = results_raw
    # Calculate cgammas_bar and mbars
    transient = int(5/6*duration / deltat) // skp
    back_components = back_pms[-1]
    basis = back_components / l2_norm(back_components, axis=1)[:, None] 
    
    # Dot products \bar{c}_{\gamma} = \bar{\vec{m}} \cdot \vec{x}_{\gamma}
    mbarser, c_gammas, cbars_gamma = compute_mbars_hgammas_hbargammas(
                                mser_ibcm, coupling_eta_ibcm, basis)
    
    # Moments of concentrations
    conc_ser = nuser_ibcm[:, :, 1]
    mean_conc = np.mean(conc_ser)
    sigma2_conc = np.var(conc_ser)
    thirdmom_conc = np.mean((conc_ser - mean_conc)**3)
    moments_conc = [float(mean_conc), float(sigma2_conc), float(thirdmom_conc)]

    # Count how many dot products are at each possible value. Use cbar = 1.0 as a split. 
    cbars_gamma_mean = np.mean(cbars_gamma[transient:], axis=0)
    specif_gammas = np.argmax(np.mean(cbars_gamma[transient:], axis=0), axis=1)
    
    cbarser_norm_centered = cbarser_ibcm - np.mean(cbarser_ibcm[transient:], axis=0)
    conc_ser_centered = conc_ser - np.mean(conc_ser[transient:], axis=0)
    correl_c_conc = np.mean(cbarser_norm_centered[transient:, :, None] 
                      * conc_ser_centered[transient:, None, :], axis=0)
    
    ysernorm_ibcm = l2_norm(yser_ibcm, axis=1)
    
    # Examples of mixing new odors with the background
    rgen = np.random.default_rng(np.random.SeedSequence(rgenseed).spawn(2)[1])
    back_pms_local = list(back_pms)
    back_pms_local[-2] = eps_ser[t_mix]
    mixres = mix_new_odors_in_manifold(back_pms_local, conc_ser, 1.0, rgen, n_ex=n_ex, n_samp=n_samp)
    mixed_samples, new_odors = mixres
    results_clean = (cbars_gamma, wser_ibcm, bkvecser_ibcm, eps_ser, ysernorm_ibcm, moments_conc, 
                     cbars_gamma_mean, specif_gammas, correl_c_conc, back_components, 
                     conc_ser, mixed_samples, new_odors)
    return results_clean

In [ ]:
# Plotting function
# Plotting functions for IBCM
def plot_ibcm_results(res_ibcm_raw, res_ibcm_clean):
    (cbars_gamma, wser_ibcm, bkvecser_ibcm, eps_ser, ysernorm_ibcm, 
         moments_conc, cbars_gamma_mean, specif_gammas, correl_c_conc, 
         back_comps, conc_ser, _, _) = res_ibcm_clean

    # Plot of cbars gamma series
    fig , ax, _ = plot_hbars_gamma_series(tser_common, cbars_gamma, 
                            skp=2, transient=320000 // skp)
    fig.tight_layout()
    leg = ax.legend(loc="upper left", bbox_to_anchor=(1., 1.))
    plt.show()
    plt.close()


    # Plots of neuron specificities
    fig, ax = plt.subplots()
    img = ax.imshow(correl_c_conc.T)
    ax.set(ylabel=r"Component $\gamma$", xlabel=r"Neuron $i$")
    fig.colorbar(img, label=r"$\langle (\bar{c}^i - \langle \bar{c}^i \rangle)"
                 r"(\nu_{\gamma} - \langle \nu_{\gamma} \rangle) \rangle$", 
                location="top")
    fig.tight_layout()
    plt.show()
    plt.close()

    # Check if each component has at least one neuron
    print("Odor specificities:", specif_gammas)
    split_val = 2.5
    for comp in range(n_components):
        print("Number of neurons specific to component {}: {}".format(
                comp, np.sum(np.mean(cbars_gamma[-2000:, :, comp], axis=0) > split_val)))

    # Plot of background inhibition
    fig, ax, bknorm_ser, ynorm_ser = plot_background_norm_inhibition(
                                    tser_common, res_ibcm_raw[2], res_ibcm_raw[8], skp=2)

    # Compute noise reduction factor, annotate
    transient = 250000 // skp
    norm_stats = compute_back_reduction_stats(bknorm_ser, ynorm_ser, trans=transient)

    print("Mean activity norm reduced to "
          + "{:.1f} % of input".format(norm_stats['avg_reduction'] * 100))
    print("Standard deviation of activity norm reduced to "
          + "{:.1f} % of input".format(norm_stats['std_reduction'] * 100))
    ax.annotate("St. dev. reduced to {:.1f} %".format(norm_stats['std_reduction'] * 100), 
               xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

    ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
    fig.tight_layout()
    
    # TODO: plot epsilon vector series?
    
    plt.show()
    plt.close()

### Function to run and clean a simulation

Uses global BioPCA parameters defined above. 

In [ ]:
def run_biopca_simulation_adapt(adapt_params, n_comp, n_dim, rgenseed, simseed, skp_local=skp):
    print("Initializing BioPCA simulation for adapt_params[:3] =", adapt_params[:3])
    # Initialize background parameters, give same rgenseed as IBCM to have same background
    rgen = np.random.default_rng(rgenseed)
    res = initialize_back_params(adapt_params, rgen, n_comp, n_dim)
    back_params_local, init_back = res
        
    init_synapses_pca = rgen.standard_normal(size=[n_i_pca, n_dim]) / np.sqrt(n_i_pca)
    init_mmat_pca = rgen.standard_normal(size=[n_i_pca, n_dim]) / np.sqrt(n_dim)
    init_lmat_pca = np.eye(n_i_pca, n_i_pca)  # Supposed to be near-identity, start as identity
    ml_inits_pca = [init_mmat_pca, init_lmat_pca]
    
    # Run the IBCM simulation
    print("Starting BioPCA simulation...")
    tstart = perf_counter()
    sim_results = integrate_biopca_adaptation(
                ml_inits_pca, update_fct, init_back, biopca_rates, 
                inhib_rates, back_params_local, adapt_params, duration, deltat, 
                seed=simseed, noisetype="uniform", skp=skp_local, **pca_options
    )
    tend = perf_counter()
    print("Finished BioPCA simulation in {:.2f} s".format(tend - tstart))
    
    return back_params_local, sim_results

In [ ]:
def analyze_clean_biopca_simul(results_raw):
    """
    We do not need to save odor vectors (back_components), 
    since the IBCM simulation will provide them for both models. 
    
    Args:
        results_raw = (tser_pca, nuser_pca, bkvecser_pca, mser_pca, 
            lser_pca, xser_pca, cbarser_pca, wser_pca, yser_pca)
    Returns:
        bkvecser_pca, ysernorm_pca, wser_pca, true_pca, 
            learnt_pca, off_diag_l_avg_abs, align_error_ser)
    """
    (tser_pca, nuser_pca, bkvecser_pca, eps_ser, mser_pca, lser_pca, xser_pca, 
         cbarser_pca, wser_pca, yser_pca) = results_raw
    
    # Analyze versus true offline PCA of the background samples
    print("Starting analysis of BioPCA vs true PCA")
    tstart = perf_counter()
    res = analyze_pca_learning(bkvecser_pca, mser_pca, lser_pca, 
                           lambda_mat_diag, demean=pca_options["remove_mean"])
    true_pca, learnt_pca, _, off_diag_l_avg_abs, align_error_ser = res
    tend = perf_counter()
    print("Completed analysis in {:.1f} s".format(tend - tstart))
    
    ysernorm_pca = l2_norm(yser_pca, axis=1)
    bkvecsernorm_pca = l2_norm(bkvecser_pca, axis=1)
    
    # Also save info about background vs yser_pca
    results_clean = (bkvecsernorm_pca, eps_ser, ysernorm_pca, wser_pca,
                     true_pca, learnt_pca, off_diag_l_avg_abs, align_error_ser)
    return results_clean

In [ ]:
def plot_biopca_results(res_biopca_raw, res_biopca_clean):
    (bkvecsernorm_pca, epser_pca, ysernorm_pca, wser_pca, true_pca, 
    learnt_pca, off_diag_l_avg_abs, align_error_ser) = res_biopca_clean

    # Plot learnt vs true PCA
    fig, axes = plot_pca_results(tser_common/1000, true_pca, learnt_pca, align_error_ser, off_diag_l_avg_abs)
    axes[-1].set_xlabel("Time (x1000 steps)")
    axes[0].get_legend().remove()
    fig.tight_layout()
    fig.set_size_inches(fig.get_size_inches()[0], 2.5*plt.rcParams["figure.figsize"][1])
    plt.show()
    plt.close()

    # Plot level of background inhibition
    fig, ax, bknorm_ser, ynorm_ser = plot_background_norm_inhibition(
                                    tser_common, res_biopca_raw[2], res_biopca_raw[9], skp=2)

    # Compute noise reduction factor, annotate
    transient = 250000 // skp
    norm_stats = compute_back_reduction_stats(bknorm_ser, ynorm_ser, trans=transient)

    print("Mean activity norm reduced to "
          + "{:.1f} % of input".format(norm_stats['avg_reduction'] * 100))
    print("Standard deviation of activity norm reduced to "
          + "{:.1f} % of input".format(norm_stats['std_reduction'] * 100))
    ax.annotate("St. dev. reduced to {:.1f} %".format(norm_stats['std_reduction'] * 100), 
               xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

    ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
    fig.tight_layout()
    plt.show()
    plt.close()

## Optimal $P$ linear manifold learning matrix
Compute average across a background time series, using the adapted $\epsilon(t)$ value going with each background sample. 

In [ ]:
def mix_new_back_adapt(back_odors, new_odors, cser, newconc, fmax, eps_ser):
    n_new = new_odors.shape[0]
    assert n_new == cser.shape[0]  # one new odor per back sample
    all_mixvecs = []
    for n in range(n_new):
        joint_concs = np.concatenate([cser[n], np.full(1, newconc)])
        joint_components = np.concatenate(
            [back_odors, new_odors[n:n+1]], axis=0)
        mixvecs = combine_fct(joint_concs, 
                    joint_components, eps_ser[n], fmax=fmax)
        all_mixvecs.append(mixvecs)
    mixvecs = np.stack(all_mixvecs, axis=0)
    return mixvecs


def get_optimal_mat_p(bkvecser, concser, eps_ser, back_pms, new_concs_rel, 
                      sd=0xdf8cc55ff9195d82ed83ae87ba4e10fc):
    """ Compute the optimal linear manifold learning matrix P, 
    using a previously simulated background"""
    avg_whiff_conc = np.mean(truncexp1_average(*back_pms[4:6]))
    new_concs = avg_whiff_conc * new_concs_rel
    osn_ampli = back_pms[-3]
    back_comp = back_pms[-1]

    # Compute optimal W matrix for all new odors possible
    # Need samples from the background (use provided bkser)
    # and samples from mixtures of background + new odor
    # (generate from back. conc. series in nuser_ibcm)
    dummy_rgen = np.random.default_rng(sd)
    # New odors, each with a subset of the background samples
    n_samp, n_dims = bkvecser.shape[0], bkvecser.shape[1]
    new_odors_from_distrib = generate_odor_tanhcdf(
        [n_samp, n_dims], dummy_rgen, unit_scale=kscale)

    optimal_matrices = []
    for newconc in new_concs:
        # Mix new odors at newconc with background
        s_new_mix = mix_new_back_adapt(back_comp, new_odors_from_distrib, 
                                 concser, newconc, osn_ampli, eps_ser)
        mat = compute_optimal_matrix_fromsamples(bkvecser, s_new_mix)
        optimal_matrices.append(mat)

    return optimal_matrices

## New odor recognition functions
For now, forget average subtraction, optimal $P$ learning, and orthogonal habituation. Just try to answer the question: when OSNs adapt, is it still necessary to habituate with manifold learning to improve new odor recognition? 

In other words, the question is not so much "does manifold learning still work with OSN adaptation?", but rather "is manifold learning still needed with OSN adaptation?". 

Compare no habituation without adaptation (setting $\epsilon$ to the average in each OSN during the simulation), no habituation with adaptation, and the IBCM / BioPCA models with adaptation. 

### Technical remarks

Since OSN response adapts over time, we test odor recognition at multiple time points, using only the background sample from the simulated instance (to which OSNs have adapted) but not extra samples (to which OSNs would not have adapted). 

In [ ]:
def find_snap_index(dt, skip, times):
    """ Find nearest multiple of dt*skip to each time in times """
    return np.around(times / (dt*skip)).astype(int)

In [ ]:
# For IBCM and BioPCA
def test_odor_recognition_adaptation(
        back_pms, new_od_kmats, ibcm_res, biopca_res, 
        opts_ibcm, opts_biopca, proj_mat, proj_args,
        rates_ibcm, rates_pca, optim_mat, 
        new_conc_rel=1.0, n_test_t=100, n_new=100):
    # Select test times, etc.
    bk_comp = back_pms[-1]
    n_comp, n_dim = bk_comp.shape[0], bk_comp.shape[1]
    n_kc = proj_mat.shape[0]
    
    # New odors tested
    new_conc = new_conc_rel * np.mean(truncexp1_average(*back_pms[4:6]))
    
    # Load background and epsilon series, assert it's the same background for both models
    conc_ser = ibcm_res[1][:, :, 1]  # concentrations
    conc_ser_pca = biopca_res[1][:, :, 1]
    assert np.allclose(conc_ser, conc_ser_pca)
    eps_ser = ibcm_res[3]
    eps_ser_pca = biopca_res[3]
    assert np.allclose(eps_ser, eps_ser_pca)
    
    # Time series of relevant weights
    mser_ibcm = ibcm_res[4]
    wser_ibcm = ibcm_res[7]
    mser_pca = biopca_res[4]
    lser_pca = biopca_res[5]
    xser_pca = biopca_res[6]
    wser_pca = biopca_res[8]
    
    # Reference tags are computed for the average epsilon of each OSN
    # TODO: consider recomputing them at each instantaneous tested epsilon, 
    # since epsilon adaptation changes the tag that the new odor alone would generate; 
    # see which is better later. 
    default_eps = np.mean(eps_ser, axis=0)  # average epsilon for each OSN
    single_mean_eps = np.full(n_dim, np.mean(default_eps))
    osn_ampli = back_pms[-3]

    # OSN response to new odors and back odors at average conc.
    new_odor_responses = np.stack([combine_fct(np.asarray([new_conc]), new_od_kmats[i:i+1], 
            default_eps, fmax=osn_ampli) for i in range(n_new)])
    back_odor_responses = np.stack([combine_fct(np.asarray([new_conc]), bk_comp[i:i+1], 
            default_eps, fmax=osn_ampli) for i in range(n_comp)])

    # Test times, based on global parameters (duration, deltat, skp)
    start_test_t = duration - 30000.0
    test_times = np.linspace(start_test_t, duration, n_test_t)
    test_times -= deltat*skp
    test_idx = find_snap_index(deltat, skp, test_times)

    # Containers for y vectors of each model in response to the mixture
    models = ["none", "adapt", "optimal_adapt", "biopca_adapt", "ibcm_adapt"]
    mixture_yvecs = {a: np.zeros([n_new, n_test_t, n_dim]) 
                    for a in models}
    mixture_tags = {a: SparseNDArray((n_new, n_test_t, n_kc), dtype=bool) 
                    for a in models}
    new_odor_tags = sparse.lil_array((n_new, n_kc), dtype=bool)
    jaccard_scores = {a: np.zeros([n_new, n_test_t]) 
                      for a in models}
    jaccard_backs = {a: np.zeros([n_new, n_test_t]) 
                      for a in models}
    back_tags = [project_neural_tag(b, b, proj_mat, **proj_args) 
                 for b in back_odor_responses]
    
    # Assess recognition of new odors mixed non-linearly
    for i in range(n_new):
        # Compute neural tag of the new odor alone, without inhibition
        new_tag = project_neural_tag(
                        new_odor_responses[i], new_odor_responses[i],
                        proj_mat, **proj_args
                    )
        new_odor_tags[i, list(new_tag)] = True

        # Now, loop over snapshots, mix the new odor with the back samples,
        # compute the PN response at each test concentration,
        # compute tags too, and save results
        # Combine background and new odor i's parameters into one joint K matrix
        # to use in the combine_fct. 
        joint_odor_kmats = np.concatenate([bk_comp, new_od_kmats[i:i+1]], axis=0)
        for j in range(n_test_t):
            current_eps = eps_ser[j]
            jj = test_idx[j]
            joint_conc_samples = np.concatenate(
                [conc_ser[j], np.full(1, new_conc)], axis=0)
            mixture = combine_fct(joint_conc_samples, joint_odor_kmats, 
                                   current_eps, fmax=osn_ampli)
            mixture_noadapt = combine_fct(joint_conc_samples, joint_odor_kmats, 
                                   single_mean_eps, fmax=osn_ampli)
            # odors, mlx, wmat, 
            # Compute for each model
            mixture_yvecs["ibcm_adapt"][i, j] = ibcm_respond_new_odors(
                mixture, mser_ibcm[jj], wser_ibcm[jj], 
                rates_ibcm, options=opts_ibcm
            )
            mixture_yvecs["biopca_adapt"][i, j] = biopca_respond_new_odors(
                mixture, [mser_pca[jj], lser_pca[jj], xser_pca[jj]], 
                wser_pca[jj], rates_pca, options=opts_biopca
            )
            mixture_yvecs["adapt"][i, j] = mixture
            mixture_yvecs["none"][i, j] = mixture_noadapt
            mixture_yvecs["optimal_adapt"][i, j] = mixture - optim_mat.dot(mixture)
            #mixture_yvecs["orthogonal"][i, j] = mixtures - mixtures.dot(back_projector.T)
            for mod in mixture_yvecs.keys():
                mix_tag = project_neural_tag(
                    mixture_yvecs[mod][i, j], mixture,
                    proj_mat, **proj_args
                )
                try:
                    mixture_tags[mod][i, j, list(mix_tag)] = True
                except ValueError as e:
                    print(mix_tag)
                    print(mixture_yvecs[mod][i, j])
                    print(proj_mat.dot(mixture_yvecs[mod][i, j]))
                    raise e
                jaccard_scores[mod][i, j] = jaccard(mix_tag, new_tag)
                jaccard_backs[mod][i, j] = max((jaccard(mix_tag, b) for b in back_tags))
    return jaccard_scores, jaccard_backs, mixture_tags, new_odor_tags

# Additional plotting functions, to visualize the manifold

In [ ]:
# Since we will make similar plots, define functions
def plot_manifold(bkser, bkvecs, conc_ser, view_params, 
                  mixed_new_odors=None, new_odor_vec=None, dims=(0, 1, 2)):
    # Plot 2D manifold in a 3D slice,
    fig = plt.figure()
    ax = fig.add_subplot(projection="3d")
    # Too many combinations for 6 odors, maybe just highlight
    # single-odor axes
    where_each = (conc_ser > 0).astype(bool)
    n_odors = where_each.shape[1]
    locations = {}
    # Track places with 0 or 1 odor
    any_single_odor = np.all(where_each == False, axis=1)  # start with places with 0 odor
    for i in range(n_odors):
        mask = np.zeros((1, n_odors), dtype=bool)
        mask[0, i] = True
        locations["Odor {}".format(i)] = np.all(where_each == mask, axis=1)
        any_single_odor += locations["Odor {}".format(i)]  # add places with odor i only
    locations["2+ odors"] = np.logical_not(any_single_odor)  # 2+ odors anywhere else
    single_odor_colors = sns.color_palette("colorblind", n_colors=n_odors)
    all_colors = {"Odor {}".format(i): single_odor_colors[i] for i in range(n_odors)}
    all_colors["2+ odors"] = "grey"
    
    orig = np.zeros([3, 6])
    locations_order = ["2+ odors"] + ["Odor {}".format(i) for i in range(n_odors)]
    for lbl in locations_order:
        alpha = 0.3 if lbl.startswith("2+") else 1.0
        slc = locations[lbl]
        tskp = 5 if lbl.startswith("2+") else 1
        zshift = 0.03 if lbl.startswith("2+") else 0.0
        shift = 0.03 if lbl.startswith("2+") else 0.0
        lbl_append = ""# if lbl.startswith("2+") else " alone"
        bk_subset = [bkser[slc, d].copy() + shift for d in dims]
        bk_subset[2] -= zshift
        ax.scatter(bk_subset[0][::tskp], bk_subset[1][::tskp], bk_subset[2][::tskp], 
                   s=4, lw=0.3, label=lbl+lbl_append, color=all_colors[lbl], alpha=alpha)
    vecs = bkvecs / l2_norm(bkvecs, axis=1)[:, None]
    print(vecs.shape)
    ax.quiver(*orig, *(vecs[:, dims].T), color="k", lw=1.5, arrow_length_ratio=0.2)
    ax.scatter(0, 0, 0, color="k", s=25)
    
    # Also show what adding a new odor can do -- out of the manifold?
    new_odor_lbl = "+ new odor"
    if mixed_new_odors is not None:
        n_new_odors = mixed_new_odors.shape[0]
        new_odors_palette = sns.dark_palette("r", n_colors=n_new_odors+1)[1:]
        for i in range(n_new_odors):
            lbl = new_odor_lbl + " {}".format("abcdefghijklmnop".upper()[i])
            all_colors[lbl] = new_odors_palette[i]
            ax.scatter(mixed_new_odors[i, :, dims[0]], mixed_new_odors[i, :, dims[1]], 
                        mixed_new_odors[i, :, dims[2]], s=6, lw=0.3, 
                        label=lbl+lbl_append, color=all_colors[lbl], alpha=1.0)
            if new_odor_vec is not None:
                vec = new_odor_vec[i] / l2_norm(new_odor_vec[i])
                ax.quiver(*orig, *(vec[list(dims)]), color=all_colors[lbl], 
                          lw=1.5, arrow_length_ratio=0.2)
    # No new odors shown
    else:
        n_new_odors = 0
        

    # Labeling
    for lbl, f in enumerate([ax.set_xlabel, ax.set_ylabel, ax.set_zlabel]):
        # z label gets caught in the zlbl variable at the last iteration
        zlbl = f("OSN {} (of {})".format(lbl+1, bkser.shape[1]), labelpad=-17.5)
    for f in [ax.set_xticks, ax.set_yticks, ax.set_zticks]:
        f([])
    for f in [ax.set_xticklabels, ax.set_yticklabels, ax.set_zticklabels]:
        f([], pad=0.1)
    view_params.setdefault("azim", 240)
    view_params.setdefault("elev", 3)
    ax.view_init(**view_params)
    handles, labels = ax.get_legend_handles_labels()
    # Move the label for 2+ odors to before the new odors
    if n_new_odors > 0:
        handles.insert(-n_new_odors-1, handles[0])
        labels.insert(-n_new_odors-1, labels[0])
    else:
        handles.append(handles[0])
        labels.append(labels[0])
    handles.pop(0)
    labels.pop(0)
    leg = ax.legend(handles=handles, labels=labels, 
        frameon=True, ncol=1, loc="upper left", bbox_to_anchor=(0.85, 1.0), 
        title="Odor presence", title_fontsize=6)
    #loc="upper right", bbox_to_anchor=(0.0, 1.0), frameon=False)
    fig.tight_layout()

    # Need to adjust the tightbox to remove whitespace above and below manually. 
    #ax.set_aspect("equal")
    fig.tight_layout()
    tightbox = fig.get_tightbbox()
    tightbox._bbox.y0 = tightbox._bbox.y0*1.1   #bottom
    tightbox._bbox.y1 = tightbox._bbox.y1 + 0.7*tightbox._bbox.y0  # top
    tightbox._bbox.x0 = tightbox._bbox.x0 * 0.6  # position of left side

    return fig, ax, tightbox

In [ ]:
# Inspect the background process
def pairplots_background(bkser, bkvecs, epsser=None, mixed_new_odors=None, new_odor_vec=None):
    # Background vectors time series with mixed concentrations
    tslice = slice(0, None, 30)
    
    # Scale odor affinities K_{i \gamma} by the average saturation threshold 
    # of the OSN, exp(\epsilon_i), to get the effective affinity scale, 
    # before normalizing each odor vector K_\gamma
    vecs = bkvecs / l2_norm(bkvecs, axis=1)[:, None]
    if epsser is not None:
        mean_eps = np.mean(epsser, axis=0)
        vecs_eff = bkvecs / np.exp(mean_eps[None, :])
        vecs_eff = vecs_eff / l2_norm(vecs_eff, axis=1)[:, None]
    else:
        vecs_eff = None
        
    n_comp = bkvecs.shape[0]
    n_cols = 6
    n_plots = 48 // 2
    n_rows = n_plots // n_cols + min(1, n_plots % n_cols)
    fig, axes = plt.subplots(n_rows, n_cols, sharex=True, sharey=True)
    fig.set_size_inches(n_cols*1.0, n_rows*1.0)
    single_odor_colors = sns.color_palette("colorblind", n_colors=n_comp)
    all_colors = {"Odor {}".format(i): single_odor_colors[i] for i in range(n_comp)}
    if mixed_new_odors is not None:
        n_new_odors = mixed_new_odors.shape[0]
        new_odors_palette = sns.dark_palette("r", n_colors=n_new_odors+1)[1:]
    for i in range(n_plots):
        ax = axes.flat[i]
        ax.scatter(bkser[tslice, 2*i+1], bkser[tslice, 2*i], 
                   s=9, alpha=0.5, color="k")
        for j in range(n_comp):
            ax.plot(*zip([0.0, 0.0], vecs[j, 2*i:2*i+2][::-1]), lw=1.5, 
                    color=single_odor_colors[j])
            if epsser is not None:
                ax.plot(*zip([0.0, 0.0], vecs_eff[j, 2*i:2*i+2][::-1]), lw=1.5, ls="--",
                    color=single_odor_colors[j])
        if mixed_new_odors is not None:
            for j in range(n_new_odors):
                clr = new_odors_palette[j]
                ax.scatter(mixed_new_odors[j, :, 2*i+1], mixed_new_odors[j, :, 2*i], 
                       s=6, alpha=1.0, color=clr)
        if new_odor_vec is not None:
            for j in range(n_new_odors):
                vec = new_odor_vec[j] / l2_norm(new_odor_vec[j])
                ax.plot(*zip([0.0, 0.0], vec[2*i:2*i+2][::-1]), lw=2.0, 
                    color=new_odors_palette[j])
        ax.set(xlabel="OSN {}".format(2*i+2), ylabel="OSN {}".format(2*i+1))
    for i in range(n_plots, n_rows*n_cols):
        axes.flat[i].set_axis_off()
    fig.tight_layout()
    
    return fig, axes

In [ ]:
# Axes zoom effect from Matplotlib documentation
from matplotlib.transforms import (Bbox, TransformedBbox,
                                   blended_transform_factory)
from mpl_toolkits.axes_grid1.inset_locator import (BboxConnector,
                                                   BboxConnectorPatch,
                                                   BboxPatch)
def connect_bbox(bbox1, bbox2,
                 loc1a, loc2a, loc1b, loc2b,
                 prop_lines, prop_patches=None):
    if prop_patches is None:
        prop_patches = {
            **prop_lines,
            "alpha": prop_lines.get("alpha", 1) * 0.2,
            "clip_on": False,
        }

    c1 = BboxConnector(
        bbox1, bbox2, loc1=loc1a, loc2=loc2a, clip_on=False, **prop_lines)
    c2 = BboxConnector(
        bbox1, bbox2, loc1=loc1b, loc2=loc2b, clip_on=False, **prop_lines)

    bbox_patch1 = BboxPatch(bbox1, **prop_patches, color="grey")
    bbox_patch2 = BboxPatch(bbox2, **prop_patches, color="grey")

    p = BboxConnectorPatch(bbox1, bbox2,
                           loc1a=loc1a, loc2a=loc2a, loc1b=loc1b, loc2b=loc2b,
                           clip_on=False,
                           **prop_patches)

    return c1, c2, bbox_patch1, bbox_patch2, p

def zoom_effect01(ax1, ax2, xmin, xmax, **kwargs):
    """
    Connect *ax1* and *ax2*. The *xmin*-to-*xmax* range in both Axes will
    be marked.

    Parameters
    ----------
    ax1
        The main Axes.
    ax2
        The zoomed Axes.
    xmin, xmax
        The limits of the colored area in both plot Axes.
    **kwargs
        Arguments passed to the patch constructor.
    """

    bbox = Bbox.from_extents(xmin, 0, xmax, 1)

    mybbox1 = TransformedBbox(bbox, ax1.get_xaxis_transform())
    mybbox2 = TransformedBbox(bbox, ax2.get_xaxis_transform())

    prop_patches = {**kwargs, "ec": "none", "alpha": 0.2}

    c1, c2, bbox_patch1, bbox_patch2, p = connect_bbox(
        mybbox1, mybbox2,
        loc1a=3, loc2a=2, loc1b=4, loc2b=1,
        prop_lines=kwargs, prop_patches=prop_patches)

    #ax1.add_patch(bbox_patch1)
    ax2.add_patch(bbox_patch2)
    ax2.add_patch(c1)
    ax2.add_patch(c2)
    ax2.add_patch(p)

    return c1, c2, bbox_patch1, bbox_patch2, p

In [ ]:
# Check what individual OSN epsilons are doing on fast and long time scales
def plot_eps_series(eps_ser, n_shown, tzoom_interv, skp_n=1, smoothsize=201):
    fig = plt.figure()
    gs = fig.add_gridspec(2, 4)
    axes = [fig.add_subplot(gs[0, :3])]
    axes.append(fig.add_subplot(gs[1, :3], sharey=axes[0]))
    axleg = fig.add_subplot(gs[:, 3])
    
    fig.set_size_inches(plt.rcParams["figure.figsize"][0], plt.rcParams["figure.figsize"][1])
    eps_ser_smooth = moving_average(eps_ser, kernelsize=smoothsize, boundary="free")
    tscale = deltat * 10.0 / 1000.0 / 60.0  # minutes

    osn_colors = sns.cubehelix_palette(n_colors=n_shown, 
            start=0.0, rot=1.0, gamma=1.0, hue=0.8, light=0.85, dark=0.15, reverse=True)

    tsl_local = slice(*tzoom_interv, 1)  # limit * skp * 10 = milliseconds, skp=50 default
    tsl_global = slice(0, None, 20)
    n_labels = 6
    skp_lbl = (n_shown // skp_n) // n_labels
    for i in range(0, n_shown, skp_n):
        lbl = "{}".format(i) if (i // skp_n) % skp_lbl == 0 else ""
        axes[0].plot(tser_common[tsl_local]*tscale, eps_ser[tsl_local, i], 
                     alpha=0.7, lw=0.75, color=osn_colors[i], label=lbl)
        axes[1].plot(tser_common[tsl_global]*tscale, eps_ser_smooth[tsl_global, i], 
                     alpha=0.7, lw=0.75, color=osn_colors[i])

    axes[0].set(ylabel=r"$\epsilon_i(t)$")
    axes[1].set(xlabel="Time (min)", ylabel=r"Smoothed $\epsilon_i(t)$")
    axleg.legend(*axes[0].get_legend_handles_labels(), frameon=False, title="OSN")
    axleg.set_axis_off()
    t1, t2 = tser_common[tzoom_interv[0]]*tscale, tser_common[tzoom_interv[1]]*tscale
    zoom_effect01(axes[0], axes[1], t1, t2, lw=0.8)
    fig.tight_layout(h_pad=-0.1)
    return fig, axes, axleg

In [ ]:
# Specific model names and colors
model_nice_names.update({
    "ibcm_adapt": "IBCM + adapt.",
    "biopca_adapt": "BioPCA + adapt.",
    "adapt": "OSN adaptation",
    "optimal_adapt": "Optim. $P$ + adapt."
})
model_colors.update({
    "ibcm_adapt": model_colors.get("ibcm"),
    "biopca_adapt": model_colors.get("biopca"),
    "adapt": "xkcd:purple",
    "optimal_adapt": model_colors.get("optimal")
})

# Plot jaccard similarities
def plot_jaccards(jac_scores):
    # Plot model histogram results for one new odor concentration
    fig, ax = plt.subplots()
    fig.set_size_inches(plt.rcParams["figure.figsize"][0]*1.3, 
                       plt.rcParams["figure.figsize"][1])
    models = [m for m in ["none", "adapt", "optimal_adapt", "biopca_adapt", "ibcm_adapt"] 
              if m in jac_scores.keys()]
    for m in models:  # Plot IBCM last
        all_jacs = jac_scores[m]
        hist_outline(
            ax, all_jacs.flatten(),
            bins="doane", density=True, label=model_nice_names.get(m, m),
            color=model_colors.get(m), alpha=1.0
        )
        ax.axvline(
            np.median(all_jacs), ls="--",
            color=model_colors.get(m)
        )
    # Labeling the graph, etc.
    ax.set_xlabel("Jaccard similarity (higher is better)")
    ax.set_ylabel("Probability density")
    ax.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0), frameon=False)
    return fig, ax

# Main simulations

## IBCM habituation and simulation parameters

In [ ]:
# IBCM model parameters, same for each tested epsilon
n_i_ibcm = 24  # Number of inhibitory neurons for IBCM case

# Model rates
learnrate_ibcm = 0.001  #5e-5
tau_avg_ibcm = 1600  # 2000
coupling_eta_ibcm = 0.7/n_i_ibcm
ssat_ibcm = 50.0
k_c2bar_avg = 0.5
decay_relative_ibcm = 0.005
lambd_ibcm = 1.0
ibcm_rates = [
    learnrate_ibcm, 
    tau_avg_ibcm, 
    coupling_eta_ibcm, 
    lambd_ibcm,
    ssat_ibcm, 
    k_c2bar_avg,
    decay_relative_ibcm 
]
ibcm_options = {
    "activ_fct": activ_function, 
    "saturation": "tanh", 
    "variant": "law", 
    "decay": True
}

## BioPCA habituation and simulation parameters

In [ ]:
# BioPCA model parameters, same for all epsilons
n_i_pca = n_components * 2  # Number of inhibitory neurons for BioPCA case

# Model rates
learnrate_pca = 1e-4  # Learning rate of M
# Choose Lambda diagonal matrix as advised in Minden et al., 2018
# but scale it up to counteract W regularization
lambda_range_pca = 0.5
lambda_max_pca = 9.0
# Learning rate of L, relative to learnrate. Adjusted to Lambda in the integration function
rel_lrate_pca = 2.0  #  / lambda_max_pca**2 
lambda_mat_diag = build_lambda_matrix(lambda_max_pca, lambda_range_pca, n_i_pca)

xavg_rate_pca = learnrate_pca
pca_options = {
    "activ_fct": activ_function, 
    "remove_lambda": False, 
    "remove_mean": True
}
biopca_rates = [learnrate_pca, rel_lrate_pca, lambda_max_pca, lambda_range_pca, xavg_rate_pca]

In [ ]:
#main_seed = 0x7170b82d905839ddda1def555e3a43508
main_seed = 0x7170b82d905839ddda1def555e3a43507
simul_seed = 0x52e7bfc4e1f58395730de6afff855abc

# IBCM
back_ibcm, res_ibcm = run_ibcm_simulation_adapt(adaptation_params, n_components,  
                         n_dimensions, main_seed, simul_seed)
res_ibcm_clean = analyze_clean_ibcm_simul(res_ibcm, back_ibcm, main_seed)

# BioPCA
back_biopca, res_biopca = run_biopca_simulation_adapt(adaptation_params, n_components,
                            n_dimensions, main_seed, simul_seed)
res_biopca_clean = analyze_clean_biopca_simul(res_biopca)

In [ ]:
plot_ibcm_results(res_ibcm, res_ibcm_clean)
plt.show()
plt.close()

In [ ]:
plot_biopca_results(res_biopca, res_biopca_clean)
plt.show()
plt.close()

In [ ]:
eps_ser_ibcm = res_ibcm[3]
fig, axes, axleg = plot_eps_series(eps_ser_ibcm, 50, (600, 840), skp_n=4, smoothsize=101)
plt.show()
plt.close()

In [ ]:
# (cbars_gamma, wser_ibcm, bkvecser_ibcm, eps_ser, ysernorm_ibcm, moments_conc, 
#                     cbars_gamma_mean, specif_gammas, correl_c_conc, back_components, 
#                     conc_ser, mixed_samples, new_odors)
bkser_ibcm = res_ibcm_clean[2]
bkvecs_ibcm = res_ibcm_clean[9]
concser_ibcm = res_ibcm_clean[10]
fig, ax, box = plot_manifold(bkser_ibcm, bkvecs_ibcm, concser_ibcm, {"azim":220, "elev":30}, dims=(0, 1, 2))
plt.show()
plt.close()

In [ ]:
fig, axes = pairplots_background(bkser_ibcm, bkvecs_ibcm, eps_ser_ibcm)
plt.show()
plt.close()

In [ ]:
# Generate new odors, projection matrix, etc. 
n_new_od = 100
rgen_od = np.random.default_rng(np.random.SeedSequence(main_seed).spawn(2).pop(1))
new_odors_kmats = generate_odor_tanhcdf([n_new_od, n_dimensions], rgen_od, unit_scale=kscale)

# Common parameters
n_kc = 1000 * n_dimensions // 25
projection_arguments = {
    "kc_sparsity": 0.05,
    "adapt_kc": True,
    "n_pn_per_kc": 3 * n_dimensions // 25,
    "project_thresh_fact": 0.1
}
proj_matrix = create_sparse_proj_mat(n_kc, n_dimensions, rgen_od)

In [ ]:
# Obtain optimal matrix for new concentration = avg whiff
optimal_matrix = get_optimal_mat_p(bkser_ibcm, concser_ibcm, eps_ser_ibcm, back_ibcm, np.ones(1))[0]

# Run odor recognition tests for 
new_conc_rel = 0.5
test_res = test_odor_recognition_adaptation(
                back_ibcm, new_odors_kmats, res_ibcm, res_biopca, 
                ibcm_options, pca_options, proj_matrix, projection_arguments,
                ibcm_rates, biopca_rates, optimal_matrix, 
                new_conc_rel=new_conc_rel, n_test_t=100, n_new=n_new_od
)
jaccard_scores, jaccard_backs, mixture_tags, new_odor_tags = test_res

In [ ]:
fig, ax = plot_jaccards(jaccard_scores)

ax.set_title(r"New conc. = {:.1f} $\langle c \rangle$".format(new_conc_rel))
fig.tight_layout()
plt.show()
plt.close()

# Save results of interest for final plotting

Example simulation runs. The odor performance recognition is tested across simulation seeds, new odors, etc. in the Python script ``supplementary_scripts/run_adaptation_performance_tests.py``. 

In [ ]:
def save_ibcm_simuls_to_disk(fname, clean_results):
    # Save cbar gamma series, background series, ynorm series
    (cbars_gamma, _, bkvecser_ibcm, eps_ser, ysernorm_ibcm, _, _, _, _, 
    back_comps, conc_ser, _, _) = clean_results
    all_saved_series = dict()
    all_saved_series["cbars_gamma_ser"] = cbars_gamma
    # For habituation and manifold plots, save back series
    # and odor components
    all_saved_series["bkvec_ser"] = bkvecser_ibcm
    all_saved_series["back_components"] = back_comps
    all_saved_series["conc_ser"] = conc_ser
    # For habituation plots, save norm of y
    all_saved_series["y_norm_ser"] = ysernorm_ibcm
    # Save epsilon dynamics
    all_saved_series["eps_ser"] = eps_ser

    np.savez_compressed(fname, **all_saved_series)
    return 0

def save_biopca_simuls_to_disk(fname, clean_results):
    # Save true and learnt PCA, that's all we really need
    true_learnt_pcas = {}
    (bkvecsernorm_pca, epser_pca, ysernorm_pca, wser_pca, true_pca, 
     learnt_pca, off_diag_l_avg_abs, align_error_ser) = clean_results
    true_learnt_pcas["true_pca_vals"] = true_pca[0]
    true_learnt_pcas["learnt_pca_vals"] = learnt_pca[0]
    true_learnt_pcas["pca_align_error"] = align_error_ser
    true_learnt_pcas["bkvec_norm_ser"] = bkvecsernorm_pca
    true_learnt_pcas["y_norm_ser"] = ysernorm_pca
    np.savez_compressed(fname, **true_learnt_pcas)
    return 0

In [ ]:
# Save
fname_ibcm = pj(outputs_folder, "saved_ibcm_simulations_adapt_osn.npz")
fname_biopca = pj(outputs_folder, "saved_biopca_simulations_adapt_osn.npz")
save_ibcm_simuls_to_disk(fname_ibcm, res_ibcm_clean)
save_biopca_simuls_to_disk(fname_biopca, res_biopca_clean)